In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['spark-3.4.0']= 'spark-3.4.0'


In [2]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{'spark-3.4.0'}-bin-hadoop3"

In [3]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$'spark-3.4.0'/$'spark-3.4.0'-bin-hadoop3.tgz
!tar xf $'spark-3.4.0'-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [78.0 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,057 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe am

In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [7]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True)
home_sales.createOrReplaceTempView("sales") 


In [8]:
home_sales

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
    SELECT 
      YEAR(date) AS YEAR,
      ROUND (AVG(price), 2) AS AVERAGE_PRICE 
    FROM sales
    WHERE bedrooms = 4      
    GROUP BY YEAR
    ORDER BY YEAR DESC   
    """).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
    SELECT 
      YEAR(date) AS YEAR,
      ROUND (AVG(price), 2) AS AVERAGE_PRICE 
    FROM sales
    WHERE bedrooms = 3
    and bathrooms = 3      
    GROUP BY YEAR(date)
    ORDER BY YEAR DESC   
    """).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [11]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
    SELECT 
      YEAR(date) AS YEAR,
      ROUND (AVG(price), 2) AS AVERAGE_PRICE 
    FROM sales
    WHERE bedrooms = 3
    and bathrooms = 3  
    and floors = 2
    and sqft_living >= 2000   
    GROUP BY YEAR(date)
    ORDER BY YEAR DESC   
    """).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    290242.99|
|2021|    296330.96|
|2020|    292289.09|
|2019|    289859.14|
+----+-------------+



In [16]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
 SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE
 FROM sales 
 WHERE price >= 350000
 """).show()

end_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|AVERAGE_PRICE|
+-------------+
|    473796.26|
+-------------+

--- 0.6486926078796387 seconds ---


In [25]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales") 


DataFrame[]

In [27]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [79]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

 spark.sql("""
 SELECT VIEW_RATING
 FROM sales 
 WHERE price >= 350000
 """).show() 


end_time = time.time() 



print("--- %s seconds ---" % (time.time() - start_time))


IndentationError: ignored

In [61]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales.write.parquet('date_built', mode='overwrite') 

In [78]:
# 11. Read the parquet formatted data.
home_sales=spark.read.parquet('date_built') 

In [63]:
# 12. Create a temporary table for the parquet data.
home_sales.createOrReplaceTempView("data_built")

In [65]:
 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: ignored

In [66]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [77]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("sales") 

False